# Name: Uday Routhu

### Assginment: Applying Advanced Transformations (Core)

### Imports 

In [54]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

In [55]:
# Load the superhero_info.csv and superhero_powers.csv files into DataFrames
info_df = pd.read_csv('input/superhero_info - superhero_info.csv')
powers_df = pd.read_csv('input/superhero_powers - superhero_powers.csv')

#### super_hero columns

In [56]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


#### super hero powers columns

In [57]:
powers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [58]:
info_df.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [59]:
powers_df.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


#### Devide Hero, Publisher column to make join for two data frames 

In [60]:
# Split the "Hero|Publisher" column into "Hero" and "Publisher" columns
info_df[['Hero', 'Publisher']] = info_df['Hero|Publisher'].str.split('|', expand=True)

In [61]:
#info_df doesn't has her0_names column so rename Hero to hero_name
info_df.rename(columns={'Hero': 'hero_names'}, inplace=True)

In [62]:
# Merge the DataFrames on the 'Name' column to combine the information
merged_df = info_df.merge(powers_df, on='hero_names', how='left')

#### I'm just removing publisher to avoid duplicate 

In [63]:
merged_df.drop(columns=['Publisher'], inplace=True)

In [64]:
# Split the "Hero|Publisher" column into "Hero" and "Publisher" columns
merged_df[['Hero', 'Publisher']] = info_df['Hero|Publisher'].str.split('|', expand=True)

In [65]:
merged_df.drop(columns=['Hero|Publisher'], inplace=True)

In [66]:
# Use str.extract to extract Height and Weight as numeric values
merged_df[['Height (numeric)', 'Weight (numeric)']] = merged_df['Measurements'].str.extract(r"'Height': '(?P<Height_numeric>\d+\.\d+) cm', 'Weight': '(?P<Weight_numeric>\d+\.\d+) kg'")

In [67]:
# Convert the extracted columns to numeric
merged_df['Height (numeric)'] = pd.to_numeric(merged_df['Height (numeric)'])
merged_df['Weight (numeric)'] = pd.to_numeric(merged_df['Weight (numeric)'])

### Removing unwanted column from combined data frame 

In [68]:
# I don't need these columns, so removing it 
merged_df.drop(columns=['Measurements'], inplace=True)
merged_df.drop(columns=['Powers'], inplace=True)
merged_df.drop(columns=['hero_names'], inplace=True)

In [69]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender            463 non-null    object 
 1   Race              463 non-null    object 
 2   Alignment         463 non-null    object 
 3   Hair color        463 non-null    object 
 4   Eye color         463 non-null    object 
 5   Skin color        463 non-null    object 
 6   Hero              463 non-null    object 
 7   Publisher         463 non-null    object 
 8   Height (numeric)  463 non-null    float64
 9   Weight (numeric)  463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 39.8+ KB


In [70]:
# Split the comma-separated powers into lists
powers_df['Powers'] = powers_df['Powers'].str.split(',')

In [71]:
# One-hot-encode the powers
encoded_powers  = powers_df['Powers'].str.join('|').str.get_dummies()

In [72]:
# Concatenate the one-hot-encoded powers with the original DataFrame
result_df = pd.concat([merged_df, encoded_powers], axis=1)

In [73]:
result_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (numeric),Weight (numeric),...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,0,0,0,0,0,0,0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,0,0,0,0,0,0,0,0,0,0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,0,0,0,0,0,0,1,0,0,0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,0,0,0,0,0,0,0,0,0,0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,0,0,0,0,0,0,0,0,0,0


#### II. Use your combined DataFrame to answer the following questions.

#### Compare the average weight of super powers who have Super Speed to those who do not.

In [74]:
# Filter heroes with and without Super Speed
super_speed_heroes = result_df[result_df['Super Speed'] == 1]
non_super_speed_heroes = result_df[result_df['Super Speed'] == 0]

# Calculate the average weight for each subset
avg_weight_super_speed = super_speed_heroes['Weight (numeric)'].mean()
avg_weight_non_super_speed = non_super_speed_heroes['Weight (numeric)'].mean()

# Compare the averages
print("Average weight of heroes with Super Speed:", avg_weight_super_speed)
print("Average weight of heroes without Super Speed:", avg_weight_non_super_speed)


Average weight of heroes with Super Speed: 113.39181286549707
Average weight of heroes without Super Speed: 113.70547945205479


#### What is the average height of heroes for each publisher?

In [75]:
# Group by 'Publisher' and calculate the average height
avg_height_by_publisher = result_df.groupby('Publisher')['Height (numeric)'].mean()

In [76]:
# Print the result
print(avg_height_by_publisher)

Publisher
DC Comics            181.923913
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.546128
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height (numeric), dtype: float64
